# Random Forest
## 범주형

In [1]:
path = '../'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### 데이터 로딩 ###

# train data loading
train_dataset_label = pd.read_csv(path + 'data_processing/dataset_01/training/label/train_dataset_label.csv')
train_dataset = pd.read_csv(path + 'data_processing/dataset_01/training/source/train_dataset_remove_nan.csv')

# train_output 을 DIAG_NM 하나의 col 만 가지는 data frame으로
train_output = pd.merge(train_dataset_label, train_dataset, on='ID')[['DIAG_NM']]

# train_input에서 'date' 와 'ID' column 제거
train_input = train_dataset.drop(train_dataset.columns[0], axis=1)
train_input = train_input.drop(train_input.columns[0], axis=1)

# test data loading
test_dataset_label = pd.read_csv(path +'data_processing/dataset_01/validation/label/val_dataset_label.csv')
test_dataset = pd.read_csv(path +'data_processing/dataset_01/validation/source/val_dataset_remove_nan.csv')

# test_output 을 DIAG_NM 하나의 col 만 가지는 data frame으로
test_output = pd.merge(test_dataset_label, test_dataset, on='ID')[['DIAG_NM']]

# test_input에서 'date' 와 'ID' column 제거
test_input = test_dataset.drop(test_dataset.columns[0], axis=1)
test_input = test_input.drop(test_input.columns[0], axis=1)

### 레이블 인코딩 ###

# 트리 기반 알고리즘에서는 숫자의 크기에 따른 중요도 차이가 없어 레이블 인코딩 OK
# 선형적 특징을 가지는 알고리즘들은 one hot encoding 통해 해결해야 함

from sklearn.preprocessing import LabelEncoder

# 라벨인코더 선언 및 Fitting
# CN : 0, Dem : 1, MCI : 2
items = ['CN', 'Dem', 'MCI']
encoder = LabelEncoder()
encoder.fit(items)

# train
train_output['DIAG_NM'] = encoder.transform(train_output['DIAG_NM'])
# display(encoder.classes_)
# display(encoder.inverse_transform(train_output['DIAG_NM']))

# test
test_output['DIAG_NM'] = encoder.transform(test_output['DIAG_NM'])

In [3]:
### RandomForestClassifier ### 

from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier(random_state=0, n_jobs=-1)

RF.fit(train_input, train_output)

print(RF.score(train_input, train_output))
print(RF.score(test_input, test_output)) # 0.6751873071837814

/var/folders/15/5dzd94697jbcyt28_f17m3zc0000gn/T/ipykernel_7004/4270459452.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  RF.fit(train_input, train_output)


1.0
0.6791538122520935


In [4]:
### RandomForest GridSearchCV ### 

from sklearn.model_selection import GridSearchCV

params ={
    'n_estimators':[10, 100],
    'max_depth':[6,8,10,12],
    'min_samples_leaf':[8,12,18],
    'min_samples_split':[8,16,20]
}

RF = RandomForestClassifier(random_state=0, n_jobs=-1)
grid_cv = GridSearchCV(RF, param_grid=params, cv=2, n_jobs=-1)
grid_cv.fit(train_input, train_output)
print(grid_cv.score(train_input, train_output))
print(grid_cv.score(test_input, test_output)) # 0.7236668135742618
print(grid_cv.best_params_) # {'max_depth': 6, 'min_samples_leaf': 8, 'min_samples_split': 8, 'n_estimators': 10}

/Users/ijeonhui/Github/dementia_analytics/venv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/ijeonhui/Github/dementia_analytics/venv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/ijeonhui/Github/dementia_analytics/venv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/ijeonhui/Github/dementia

0.6510868341029395
0.7236668135742618
{'max_depth': 6, 'min_samples_leaf': 8, 'min_samples_split': 8, 'n_estimators': 10}


/Users/ijeonhui/Github/dementia_analytics/venv/lib/python3.10/site-packages/sklearn/model_selection/_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)
